In [2]:
import pandas as pd
import os

In [3]:
# Read the problems file
problems = pd.read_json('code4bench_no_loops.json')

In [5]:
# For each of the problems make a folder with the index
for i, problem in problems.iterrows():
    # define the range of problems to extract here
    if i > 2000 and i < 2200:
        # Get the context of the problem
        context = problem['context']
        
        # Get the path, start with the current directory and add the folder name
        path = os.path.join(os.getcwd(), "..", "no_loop_problems", f"{i}")

        # Create the folder if it does not exist
        if not os.path.exists(path):
            os.makedirs(path)
        
        # Write the context to a file
        with open(os.path.join(path, 'context.txt'), 'w', encoding="utf-8") as f:
            f.write(context)

In [9]:
index = 56
print(problems.iloc[index])
problems.iloc[index]['sourceCode'].replace("\r", " ").replace("\n", " ").replace("\t", " ")

id                                                         2175
sourceCode    #include<stdio.h>\r\n#include<stdlib.h>\r\nint...
fullname                                                    32C
name                                                          C
countline                                                    41
context       C. Flea time limit per test 2 seconds memory l...
Name: 1657, dtype: object


'#include<stdio.h>  #include<stdlib.h>  int main()  {      long long int n,m,s,i,j,ans,c=0,d=0,f,e;      scanf("%lld %lld %lld",&n,&m,&s);      if(m/s==0||m==s){          c=m;          e=m;      }      else if(m/s<=2){          c=2;e=s+1;      }      else{          if(m%s==0){              c=(m/s-1)+1;e=(m/s-1)*s+1;          }          else {              c=(m/s)+1;e=(m/s)*s+1;          }      }      if(n/s==0||n==s){          d=n;          f=n;      }      else if(n/s<=2){          d=2;          f=s+1;      }      else{          if(n%s==0){              d=(n/s-1)+1;f=(n/s-1)*s+1;          }          else {              d=(n/s)+1;f=(n/s)*s+1;          }      }      ans=c*d*(m-e+1)*(n-f+1);      printf("%lld",ans);      return 0;  }'

In [3]:
# Add the index to the problems, starting from 0 and going to the length of the problems
problems['index'] = range(len(problems))

problems

,id,sourceCode,fullname,name,countline,context,index
0,2,#include<stdio.h>\r\nint main()\r\n{\r\n in...,581A,A,15,"<div class=""problemindexholder"" problemindex=""...",0
30,66,#include <stdio.h>\r\r\nint main()\r\r\n{\r\r\...,591A,A,19,"<div class=""problemindexholder"" problemindex=""...",1
38,69,#include<stdio.h>\r\r\nint main()\r\r\n{\r\r\n...,592B,B,15,"<div class=""problemindexholder"" problemindex=""...",2
58,102,#include<stdio.h>\r\r\nint main()\r\r\n{\r\r\n...,599A,A,51,"<div class=""problemindexholder"" problemindex=""...",3
73,159,#include<stdio.h>\r\r\nint main()\r\r\n{\r\r\n...,611A,A,23,"<div class=""problemindexholder"" problemindex=""...",4
...,...,...,...,...,...,...,...
2246,2681,#include<stdio.h>\r\r\n\r\r\nint main(void)\r\...,784A,A,17,"<div class=""problemindexholder"" problemindex=""...",72
2312,2810,#include<stdio.h>\r\r\nint main()\r\r\n{\r\r\n...,804A,A,13,"<div class=""problemindexholder"" problemindex=""...",73
2317,2819,#include<stdio.h>\r\nint main()\r\n{\r\n in...,805A,A,11,"<div class=""problemindexholder"" problemindex=""...",74
2347,2883,#include<stdio.h>\r\n#include<math.h>\r\n#incl...,817A,A,22,"<div class=""problemindexholder"" problemindex=""...",75
